This script shows an example of a working spatiotemporal visualization that shows the following data:
- Acoustic (real dates, real locations)
- Aerial surveys (real dates, two fake locations)
- Zooplankton surveys (real dates, one fake location)

In [11]:
import altair as alt
import pandas as pd
import geopandas as gpd

In [16]:
# concat3 was created in SpatioTemporal_March15.ipynb script 
    # (combines acoustic data, plus TL and Zooplnk survey dates Rob sent me on 3/8 and 3/9)
        # 3/9 email in '2019 Zooplankton Data' thread
        # 3/8 email in 'survey tracts and times' thread

concat3 = pd.read_csv('../scratch/data/concat3_demo.csv',
                              parse_dates = ['between_days'])

concat3 # each record is a date associated with a datatype we have
        #each data type has its own record for each day it's available

,Unnamed: 0,between_days,depYear,c_uniqueUnitID,latitudeDeployed_DecDeg,longitudeDeployed_DecDeg,DataType,Unnamed: 7
0,5,2011-02-17,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0211_FD020...,41.971100,-70.476500,Noise,NaN
1,2,2011-02-17,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0207_FD020...,41.894800,-70.443900,Noise,NaN
2,0,2011-02-17,2011,NaN,41.970000,-70.430000,Zooplankton,NaN
3,4,2011-02-17,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0209_FD020...,41.950800,-70.390100,Noise,NaN
4,7,2011-02-17,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0213_FD020...,41.876200,-70.341200,Noise,NaN
...,...,...,...,...,...,...,...,...
3735,36,2018-05-29,2018,2018_BRP_CCB_S1074_Dep27_20180213_PU0214_FD03182,41.963516,-70.203483,Noise,NaN
3736,35,2018-05-30,2018,2018_BRP_CCB_S1074_Dep27_20180213_PU0207_FD03184,41.967430,-70.255260,Noise,NaN
3737,37,2018-05-30,2018,2018_BRP_CCB_S1074_Dep27_20180213_PU0222_FD03178,41.937667,-70.237983,Noise,NaN
3738,34,2018-05-30,2018,2018_BRP_CCB_S1074_Dep27_20180213_PU0202_FD03186,41.956650,-70.233150,Noise,NaN


### Plotting

In [17]:
# clipped shapefile
clipped_shp = '/Users/cristiana/Documents/Duke/MP/Python/Scripting/scratch/data/newengland_clipped (1)/newengland_clipped.shp'
clip = gpd.read_file(clipped_shp).to_crs('epsg:4326')
clip.head()

,FIPS,NAME,ACRES,Shape_Leng,Shape_Area,geometry
0,25,MASSACHUSETTS,5104241.5,609872.80019,2.503760e+09,"POLYGON ((-70.82491 42.26034, -70.78642 42.234..."


In [18]:
# Massachusetts plotted with altair
alt.Chart(clip).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project('mercator')

alt.Chart(...)

In [31]:
# full interactive visual -- acoustic + aerial + zooplankton

# colors
domain = ['Noise', 'Whale', 'Zooplankton']
range_ = ['#2E86C1', '#D35400', '#27AE60']

interval = alt.selection(type='interval', encodings=['x']) 
# interactive piece
    # different types
    # interactivity along x axis
# selections can have conditions

timeline_base = alt.Chart(concat3).mark_rect().encode(
    y = alt.Y('DataType:O', axis=alt.Axis(title='Data Type')),
    color = 'DataType:N'
).properties(
    width = 600
)

timeline_overview = timeline_base.encode(
    x = alt.X(
        'between_days:T', 
        timeUnit = 'yearmonthdate', 
        axis = alt.Axis(title='Date')
    )
).add_selection( # adding interactivity
    interval
).properties(
    height = 40
)

timeline_detail = timeline_base.encode(
    x = alt.X(
        'between_days:T', 
        timeUnit='yearmonthdate',
        axis = alt.Axis(title=''),
        scale = alt.Scale(domain=interval) # using the interactive selection to show X range
    )
).properties(
    height = 100
)

basemap = alt.Chart(clip).mark_geoshape(
    fill = 'lightgray', stroke='#706545', strokeWidth=0.5
).project('mercator').properties(
    width = 600,
    height = 300
)

points = alt.Chart(concat3).mark_point().encode(
    longitude = 'longitudeDeployed_DecDeg:Q',
    latitude = 'latitudeDeployed_DecDeg:Q',
    opacity = alt.value(1),
    color = alt.Color('DataType',scale=alt.Scale(domain=domain,range=range_))
).transform_filter(
    interval
).project("mercator").properties( # can put scale parameter
    width = 600,
    height = 300
)

March19 = alt.vconcat((basemap + points), timeline_detail, timeline_overview)
March19

#March19.save('/Users/cristiana/Documents/Duke/MP/Python/Outputs/Demo2015.html')

#color=alt.Color('species', scale=alt.Scale(scheme='dark2'))



what do we need to see?
- the full temporal extent of each dataset
- which days the datasets overlap
- locations of detections

what outputs do we need?
- overlap days (reported number? csv list of dates? visual heatmap?)
- spatial proximity (how close is detection A to detection B?)